In [1]:
import pandas as pd
import numpy as np
import os

## function testing

In [2]:
def rename_duplicates(Gemeinde:str,Landkreis:str,name_duplicates:list) -> str:
    if Gemeinde in name_duplicates:
        return f'{Gemeinde} ({Landkreis})'
    else:
        return Gemeinde


def first_setup() -> pd.DataFrame:
    ############# MOBILE:
    mobile = pd.read_excel('../data/external/bba_06_2023.xlsx', sheet_name='Fläche', skiprows=2, header=1)
    mobile = mobile[(mobile['Land']=='Freistaat Bayern') & (mobile['Verwaltungsebene']!='2 - Land') & (~mobile['Name'].str.contains('Bezirk'))].reset_index(drop=True)

    # Landkreis-Code auf einheitliches Format bringen
    mobile['code_fill'] = mobile['AGS'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))
    mobile['Name'] = mobile['Name'].apply(lambda x: x.replace('Gemeinde ','').replace('Landkreis ',''))

    # Alle Doppelungen löschen -> Kreisfreie Städte tauchen nur in der Liste für Gemeinde/Städte auf
    mobile = mobile.sort_values('Verwaltungsebene',ascending=False,ignore_index=True)
    mobile.drop_duplicates('code_fill',ignore_index=True,inplace=True)

    # Jeder Gemeinde wird ihr Landkreis zugeordnen
    LK = mobile[mobile['Verwaltungsebene']=='3 - Kreis']
    LK_dict = LK[['code_fill','Name']].set_index('code_fill').to_dict()['Name']
    mobile['LK_code'] = mobile['code_fill'].apply(lambda x: int(str(x)[:3]+'000'))
    mobile['Landkreis'] = mobile['LK_code'].map(LK_dict)

    # Suche nach allen mehrfach benannten Gemeinden und bei Bedarf Hinzufügen des Landkreises
    name_counts = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde']['Name'].value_counts()
    name_duplicates = name_counts[name_counts>1].keys()
    mobile['Name'] = mobile.apply(lambda x: rename_duplicates(x.Name,x.Landkreis,name_duplicates),axis=1)

    # Datenfilter
    mobile = mobile[['code_fill','Name','Verwaltungsebene','2G','4G','5G']]

    return mobile

In [3]:
def mobile_data(code, mobile):
    mobile_i = mobile[mobile['code_fill']==code].reset_index(drop=True)
    
    export_mobile = {'2G':round(mobile_i['2G'][0],1), 
                     '4G':round(mobile_i['4G'][0],1), 
                     '5G':round(mobile_i['5G'][0],1), 
                     'link':'https://gigabitgrundbuch.bund.de/GIGA/DE/MobilfunkMonitoring/Downloads/start.html'}
    return export_mobile

In [15]:
def kfz_data(code:int) -> dict:
    # Alle Dateien im Ordner "KFZ" werden eingelesen
    Kfz_path = '../data/external/KFZ/'
    Kfz_files = os.listdir(Kfz_path)

    # Da KFZ-Daten nur auf Kreisebene existieren, wird der Code auf Kreishierarchie gemappt
    code_lk = int(str(code)[:3]+'000')
    Kfz = pd.DataFrame([])

    for file in Kfz_files:
        Kfz_i = pd.read_csv(Kfz_path + file,encoding='latin',delimiter=';')
        # Landkreis-Code auf einheitliches Format bringen
        Kfz_i['code_fill'] = Kfz_i['1_Auspraegung_Code'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))
        Kfz_i['Jahr'] = Kfz_i['Zeit'].apply(lambda x: x.split('.')[-1])
        Kfz_i = Kfz_i[['code_fill','Jahr','2_Auspraegung_Label','PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl']][Kfz_i['code_fill']==code_lk].reset_index(drop=True)
        Kfz = pd.concat([Kfz,Kfz_i],ignore_index=True,axis=0)

    Kfz.sort_values('Jahr',ignore_index=True,inplace=True)

    # Die benötigten Daten aus dem Dataframe extrahieren
    years = list(Kfz['Jahr'].unique())
    benzin = list(Kfz[Kfz['2_Auspraegung_Label']=='Benzin']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
    diesel = list(Kfz[Kfz['2_Auspraegung_Label']=='Diesel']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
    hybrid = list(Kfz[Kfz['2_Auspraegung_Label']=='Hybrid']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
    elektro = list(Kfz[Kfz['2_Auspraegung_Label']=='Elektro']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])

    gesamt = Kfz[(Kfz['Jahr']==Kfz['Jahr'].max()) & (Kfz['2_Auspraegung_Label']=='Insgesamt')].reset_index(drop=True)['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'][0]

    percentage = {"Benzin":round(benzin[-1]/gesamt*100), "Diesel":round(diesel[-1]/gesamt*100), "Hybrid":round(hybrid[-1]/gesamt*100), "Elektro":round(elektro[-1]/gesamt*100)}

    export_kfz = {"Jahr":years, 
                  "Benzin":benzin, 
                  "Diesel":diesel, 
                  "Hybrid":hybrid, 
                  "Elektro":elektro, 
                  "Anteil":percentage, 
                  "link":"https://open.bydata.de/datasets/46251-003-d?locale=de"}
    return export_kfz

In [4]:
mobile = first_setup()

In [269]:
Landkreise = mobile[mobile['Verwaltungsebene']=='3 - Kreis'][['code_fill','Name']].set_index('code_fill').to_dict()['Name']
Gemeinden = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde'][['code_fill','Name']].set_index('code_fill').to_dict()['Name']

In [5]:
export_4 = mobile_data(171111,mobile)
data_export = {'Mobilfunk':export_4}

In [17]:
kfz_data(171111)['Jahr']

['2018', '2019', '2020', '2021', '2022', '2023', '2024']

: 

In [10]:
import json
with open(f'../data/processed/chart_data_{"171111"}.json', 'w') as fp:
                json.dump(data_export, fp)

In [275]:
LK_codes = list(Landkreise.keys())
town_codes = list(Gemeinden.keys())

In [295]:
errors = []
for code in LK_codes:
    try:
        result = mobile_data(code, mobile)
    except:
        errors.append(code)

for code in town_codes:
    try:
        result = mobile_data(code, mobile)
    except:
        errors.append(code)        

errors = list(set(errors))

mobile[mobile['code_fill'].isin(errors)]

,code_fill,Name,Verwaltungsebene,2G,4G,5G


## Actual playground

In [33]:
def LK_mapping(code,LK_dict):
    try:
        return LK_dict[int(str(code)[:3]+'000')]
    except:
        pass

In [29]:
def rename_duplicates(Gemeinde:str,Landkreis:str,name_duplicates:list) -> str:
    if Gemeinde in name_duplicates:
        return f'{Gemeinde} ({Landkreis})'
    else:
        return Gemeinde

In [260]:
mobile = pd.read_excel('../data/external/bba_06_2023.xlsx', sheet_name='Fläche', skiprows=2, header=1)
mobile = mobile[(mobile['Land']=='Freistaat Bayern') & (mobile['Verwaltungsebene']!='2 - Land') & (~mobile['Name'].str.contains('Bezirk'))].reset_index(drop=True)

# Landkreis-Code auf einheitliches Format bringen
mobile['code_fill'] = mobile['AGS'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))
mobile['Name'] = mobile['Name'].apply(lambda x: x.replace('Gemeinde ','').replace('Landkreis ',''))

# Alle Doppelungen löschen -> Kreisfreie Städte tauchen nur in der Liste für Gemeinde/Städte auf
mobile = mobile.sort_values('Verwaltungsebene',ascending=False,ignore_index=True)
mobile.drop_duplicates('code_fill',ignore_index=True,inplace=True)

# Jeder Gemeinde wird ihr Landkreis zugeordnen
LK = mobile[mobile['Verwaltungsebene']=='3 - Kreis']
LK_dict = LK[['code_fill','Name']].set_index('code_fill').to_dict()['Name']
mobile['LK_code'] = mobile['code_fill'].apply(lambda x: int(str(x)[:3]+'000'))
mobile['Landkreis'] = mobile['LK_code'].map(LK_dict)

# Suche nach allen mehrfach benannten Gemeinden und bei Bedarf Hinzufügen des Landkreises
name_counts = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde']['Name'].value_counts()
name_duplicates = name_counts[name_counts>1].keys()
mobile['Name'] = mobile.apply(lambda x: rename_duplicates(x.Name,x.Landkreis,name_duplicates),axis=1)

# Datenfilter
mobile = mobile[['code_fill','Name','Verwaltungsebene','2G','4G','5G']]
mobile.head(3)

,code_fill,Name,Verwaltungsebene,2G,4G,5G
0,471455,Gemeindefreies Gebiet Hauptsmoor,4 - Gemeinde,100.0,100.0,97.98
1,574465,Gemeindefreies Gebiet Zerzabelshofer Forst,4 - Gemeinde,100.0,100.0,98.55
2,575143,Marktbergel,4 - Gemeinde,100.0,100.0,93.26


In [98]:
mobile[mobile['Name']=='Garmisch-Partenkirchen']

,code_fill,Name,Verwaltungsebene,2G,4G,5G
1578,180117,Garmisch-Partenkirchen,4 - Gemeinde,93.66,81.81,41.48
2253,180000,Garmisch-Partenkirchen,3 - Kreis,93.06,83.71,57.40


In [101]:
EE_pie[EE_pie['Verwaltungseinheit']==180000]

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²)
2283,Garmisch-Partenkirchen,SRID=31468;POINT (4434612.161451929 5269023.21...,180000,Landkreis,unbekannt,27.9,0.0,34.9,37.2,0.0,31431.0,0.0,39354.0,41914.0,0.0,112699.0,88232.0,1012.0


In [261]:
mobile[mobile['Name'].str.contains('Hausen')]

,code_fill,Name,Verwaltungsebene,2G,4G,5G
160,673129,Hausen (Rhön-Grabfeld),4 - Gemeinde,100.0,98.47,59.68
316,474134,Hausen (Forchheim),4 - Gemeinde,100.0,100.00,93.37
993,676128,Hausen (Miltenberg),4 - Gemeinde,100.0,100.00,98.62
1152,679143,Hausen b. Würzburg,4 - Gemeinde,100.0,100.00,99.32
2073,273125,Hausen (Kelheim),4 - Gemeinde,100.0,99.02,95.86


In [111]:
mobile[mobile['Name'].str.contains('Gemeindefreies Gebiet Heinersreuther Forst')]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill,LK_code,Landkreis
425,9472458,Gemeindefreies Gebiet Heinersreuther Forst (Ba...,4 - Gemeinde,Freistaat Bayern,Landkreis Bayreuth,3 - ländlich,100.0,98.80,47.47,72.93,...,86.53,16.80,42.27,100.00,75.20,38.13,7.07,472458,472000,Bayreuth
1745,9374451,Gemeindefreies Gebiet Heinersreuther Forst (Ne...,4 - Gemeinde,Freistaat Bayern,Landkreis Neustadt a.d. Waldnaab,3 - ländlich,100.0,93.23,81.28,73.44,...,74.69,15.51,15.86,98.22,92.51,81.28,0.00,374451,374000,Neustadt a.d. Waldnaab


In [123]:
EE_pie = pd.read_csv('../data/external/EA-B Recherche-Ergebnis_EE-Anteile.csv')
EE_pie = EE_pie[EE_pie['Verwaltungsebene']!='Regierungsbezirk'].sort_values('Verwaltungsebene',ignore_index=True)
EE_pie.drop_duplicates('Verwaltungseinheit',ignore_index=True,inplace=True)

#EE_i = EE_pie[EE_pie['Verwaltungseinheit']==code].reset_index(drop=True)

In [126]:
town_codes = mobile[mobile['Verwaltungsebene']=='4 - Gemeinde']['code_fill'].unique()
LK_codes = mobile[mobile['Verwaltungsebene']=='3 - Landkreis']['code_fill'].unique()

In [132]:
errors = []

for code in LK_codes:
    try:
        test_value = EE_pie[EE_pie['Verwaltungseinheit']==code].reset_index(drop=True)['Einwohnerzahl'][0]
    except:
        errors.append(code)

In [133]:
errors

[]

In [26]:
name_counts = EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']['Name'].value_counts()
name_duplicates = name_counts[name_counts>1].keys()

In [34]:
LK = energy[energy['Verwaltungsebene']=='Landkreis']
LK_dict = LK[['Verwaltungseinheit','Name']].set_index('Verwaltungseinheit').to_dict()['Name']
EE_pie['Landkreis'] = EE_pie['Verwaltungseinheit'].apply(lambda x: LK_mapping(x,LK_dict).split(' ')[0])

In [39]:
EE_pie['Name'] = EE_pie.apply(lambda x: rename_duplicates(x.Name,x.Landkreis,name_duplicates),axis=1)

In [40]:
EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']['Name'].value_counts()

Name
Winkelhaid (Nürnberger)    2
Leinburg (Nürnberger)      2
Fichtelberg (Bayreuth)     2
Ingolstadt                 1
Gaimersheim                1
                          ..
Hohenaltheim               1
Harburg (Schwaben)         1
Hainsfarth                 1
Fünfstetten                1
Kirchdorf a.Inn            1
Name: count, Length: 2226, dtype: int64

In [59]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324 entries, 0 to 2323
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Verwaltungseinheit               2324 non-null   int64  
 1   Name                             2324 non-null   object 
 2   Verwaltungsebene                 2324 non-null   object 
 3   Anteil EE am Stromverbrauch (%)  2324 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 72.8+ KB


In [176]:
Bevoelkerung = pd.read_csv('../data/external/12411-001_flat.csv',encoding='latin',delimiter=';')

# Landkreis-Code auf einheitliches Format bringen
Bevoelkerung['code_fill'] = Bevoelkerung['1_Auspraegung_Code'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))

# Spalten anpassen
Bevoelkerung['Jahr'] = Bevoelkerung['Zeit'].apply(lambda x: int(x.split('.')[-1]))
Bevoelkerung.sort_values('Jahr',ignore_index=True,inplace=True)
Bevoelkerung['BEVSTD__Bevoelkerung__Anzahl'] = Bevoelkerung['BEVSTD__Bevoelkerung__Anzahl'].apply(lambda x: int(x.replace('-','0')))

# Datenfilter
Bevoelkerung = Bevoelkerung[['code_fill','Jahr','BEVSTD__Bevoelkerung__Anzahl']]

Bevoelkerung.head()

,code_fill,Jahr,BEVSTD__Bevoelkerung__Anzahl
0,0,2013,12604244
1,576141,2013,2873
2,576137,2013,6812
3,576132,2013,7305
4,576128,2013,2787


In [184]:
years = list(Bevoelkerung['Jahr'].unique())
Bevoelkerung_i = Bevoelkerung[Bevoelkerung['code_fill']==code].reset_index(drop=True)
counts = Bevoelkerung_i['BEVSTD__Bevoelkerung__Anzahl'].to_list()
trends = [j-i for i, j in zip(counts[:-1], counts[1:])]

In [190]:
len(trends)

10

In [174]:
errors = []

for code in LK_codes:
    try:
        test_value = Bevoelkerung[Bevoelkerung['code_fill']==code].reset_index(drop=True)['BEVSTD__Bevoelkerung__Anzahl'][0]
    except:
        errors.append(code)

In [175]:
errors

[]

In [171]:
population_errors = list(set(errors))

In [173]:
mobile[(mobile['code_fill'].isin(population_errors)) & ~(mobile['Name'].str.contains('Gemeindefrei'))]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill,LK_code,Landkreis


In [191]:
mobile[mobile['code_fill']==code]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill,LK_code,Landkreis
2227,9277117,Egglham,4 - Gemeinde,Freistaat Bayern,Landkreis Rottal-Inn,3 - ländlich,100.0,98.17,79.33,92.77,...,94.71,49.82,64.14,98.25,87.56,74.61,65.48,277117,277000,Rottal-Inn


In [240]:
mobile[mobile['Name'].str.contains('Bamberg')]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill,LK_code,Landkreis
1836,9471111,Altendorf (Bamberg),4 - Gemeinde,Freistaat Bayern,Landkreis Bamberg,2 - halbstädtisch,100.00,100.00,92.32,99.41,...,99.76,91.84,97.40,100.00,97.87,89.24,24.59,471111,471000,Bamberg
1840,9461000,Stadt Bamberg,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt Bamberg,1 - städtisch,100.00,99.65,92.52,98.65,...,98.47,90.44,95.98,100.00,99.38,80.48,66.81,461000,461000,NaN
2265,9471,Bamberg,3 - Kreis,Freistaat Bayern,Landkreis Bamberg,NaN,99.91,97.69,72.49,92.92,...,94.23,54.14,77.96,96.47,85.64,59.18,22.05,471000,471000,Bamberg


In [222]:
Kfz_path = '../data/external/KFZ/'
Kfz_files = os.listdir(Kfz_path)

In [248]:
code_lk = int(str(461000)[:3]+'000')
Kfz = pd.DataFrame([])

for file in Kfz_files:
    Kfz_i = pd.read_csv(Kfz_path + file,encoding='latin',delimiter=';')
    Kfz_i['code_fill'] = Kfz_i['1_Auspraegung_Code'].apply(lambda x: int(str(x)[1:].ljust(6, '0')))
    Kfz_i['Jahr'] = Kfz_i['Zeit'].apply(lambda x: int(x.split('.')[-1]))
    Kfz_i = Kfz_i[['code_fill','Jahr','2_Auspraegung_Label','PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl']][Kfz_i['code_fill']==code_lk].reset_index(drop=True)
    Kfz = pd.concat([Kfz,Kfz_i],ignore_index=True,axis=0)

Kfz.sort_values('Jahr',ignore_index=True,inplace=True)
Kfz.tail()

,code_fill,Jahr,2_Auspraegung_Label,PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl
51,461000,2024,Insgesamt,41314
52,461000,2024,Benzin,24589
53,461000,2024,Diesel,12723
54,461000,2024,Gas,319
55,461000,2024,Hybrid,2382


In [243]:
years = list(Kfz['Jahr'].unique())
benzin = list(Kfz[Kfz['2_Auspraegung_Label']=='Benzin']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
diesel = list(Kfz[Kfz['2_Auspraegung_Label']=='Diesel']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
hybrid = list(Kfz[Kfz['2_Auspraegung_Label']=='Hybrid']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])
elektro = list(Kfz[Kfz['2_Auspraegung_Label']=='Elektro']['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'])

In [254]:
gesamt = Kfz[(Kfz['Jahr']==Kfz['Jahr'].max()) & (Kfz['2_Auspraegung_Label']=='Insgesamt')].reset_index(drop=True)['PKW__Personenkraftwagen_nach_Kraftstoffarten__Anzahl'][0]

In [255]:
benzin[-1]

24589

In [259]:
mobile['code_fill'].value_counts()

code_fill
471455    1
173137    1
172118    1
172122    1
173145    1
         ..
779163    1
779162    1
779155    1
779154    1
577000    1
Name: count, Length: 2300, dtype: int64

In [6]:
energy = pd.read_csv('../data/external/EA-B Recherche-Ergebnis_Anteil_am_Verbrauch.csv')
EE_pie = pd.read_csv('../data/external/EA-B Recherche-Ergebnis_EE-Anteile.csv')
EE_pie = EE_pie[EE_pie['Verwaltungsebene']!='Regierungsbezirk'].reset_index(drop=True)

In [15]:
energy = energy[['Verwaltungseinheit','Name','Verwaltungsebene','Anteil EE am Stromverbrauch (%)']]
energy.head()

,Verwaltungseinheit,Name,Verwaltungsebene,Anteil EE am Stromverbrauch (%)
0,161000,Ingolstadt,Gemeinde,12.5
1,161000,Ingolstadt,Landkreis,12.5
2,162000,München (Stadt),Landkreis,2.0
3,162000,München,Gemeinde,2.0
4,163000,Rosenheim,Gemeinde,34.3


In [159]:
missing_population = Bevoelkerung[Bevoelkerung['BEVSTD__Bevoelkerung__Anzahl']==0]['code_fill'].unique()

In [160]:
mobile[mobile['code_fill'].isin(missing_population)]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill,LK_code,Landkreis


In [67]:
EE_pie[EE_pie['Name'].isin(name_duplicates)].apply(lambda x: rename_duplicates(x.Name,x.Landkreis,name_duplicates),axis=1)

16            Kastl (Altötting)
24         Stammham (Altötting)
56              Eurasburg (Bad)
110        Emmering (Ebersberg)
121         Buxheim (Eichstätt)
                 ...           
2218       Lauben (Unterallgäu)
2266      Holzheim (Donau-Ries)
2284          Rain (Donau-Ries)
2306    Haldenwang (Oberallgäu)
2309        Lauben (Oberallgäu)
Length: 100, dtype: object

In [26]:
EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']['Verwaltungseinheit'].nunique()

2229

In [44]:
LK = energy[energy['Verwaltungsebene']=='Landkreis']
LK_dict = LK[['Verwaltungseinheit','Name']].set_index('Verwaltungseinheit').to_dict()['Name']
EE_pie['Landkreis'] = EE_pie['Verwaltungseinheit'].apply(lambda x: LK_mapping(x,LK_dict).split(' ')[0])

In [36]:
EE_pie['Name'].value_counts()

Name
Hausen           4
Schönberg        3
Tiefenbach       3
Holzheim         3
Kastl            3
                ..
Loitzendorf      1
Leiblfing        1
Laberweinting    1
Konzell          1
Kempter Wald     1
Name: count, Length: 2224, dtype: int64

In [45]:
EE_pie.head(2)

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²),Landkreis
0,Ingolstadt,SRID=31468;POINT (4457728.797577065 5403236.78...,161000,Gemeinde,10.0,44.6,0.0,27.1,28.3,0.0,90154.0,0.0,54769.0,57223.0,0.0,202146.0,138016.0,133.0,Ingolstadt
1,Ingolstadt,SRID=31468;POINT (4455613.551440659 5402131.15...,161000,Landkreis,unbekannt,44.6,0.0,27.1,28.3,0.0,90154.0,0.0,54769.0,57223.0,0.0,202146.0,138016.0,133.0,Ingolstadt


In [60]:
energy.head(5)

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil EE am Stromverbrauch (%),Stromverbrauch (MWh/a),Einwohnerzahl,Fläche (km²),Stand
0,Ingolstadt,SRID=31468;POINT (4457728.797577065 5403236.78...,161000,Gemeinde,10.0,12.5,1616813.0,141029.0,133.0,31.12.2022
1,Ingolstadt,SRID=31468;POINT (4455613.551440659 5402131.15...,161000,Landkreis,unbekannt,12.5,1616813.0,141029.0,133.0,31.12.2022
2,München (Stadt),SRID=31468;POINT (4466431.185606031 5335113.93...,162000,Landkreis,unbekannt,2.0,7172096.0,1512491.0,311.0,31.12.2022
3,München,SRID=31468;POINT (4468515.3765645055 5333310.3...,162000,Gemeinde,14.0,2.0,7172096.0,1512491.0,311.0,31.12.2022
4,Rosenheim,SRID=31468;POINT (4509448.34360067 5301963.408...,163000,Gemeinde,18.0,34.3,279807.0,64403.0,37.2,31.12.2022


In [31]:
energy['Name'].nunique()

2217

In [28]:
mobile[mobile['Name']=='Gemeinde Hausen']

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,5G.1,2G.2,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,code_fill
710,9273125,Gemeinde Hausen,4 - Gemeinde,Freistaat Bayern,Landkreis Kelheim,3 - ländlich,100.0,99.02,93.42,95.86,...,69.83,100.00,97.76,84.31,89.60,100.00,96.61,86.78,2.33,273125
1266,9474134,Gemeinde Hausen,4 - Gemeinde,Freistaat Bayern,Landkreis Forchheim,2 - halbstädtisch,100.0,100.00,99.02,93.37,...,64.23,100.00,98.87,86.14,93.15,100.00,100.00,98.95,39.01,474134
1728,9673129,Gemeinde Hausen,4 - Gemeinde,Freistaat Bayern,Landkreis Rhön-Grabfeld,3 - ländlich,100.0,98.47,36.69,59.68,...,23.52,99.71,90.80,32.03,55.01,96.86,86.75,8.30,8.05,673129
1835,9676128,Gemeinde Hausen,4 - Gemeinde,Freistaat Bayern,Landkreis Miltenberg,2 - halbstädtisch,100.0,100.00,51.51,98.62,...,0.00,100.00,96.61,51.51,98.62,100.00,95.10,0.00,0.00,676128


In [35]:
EE_pie[EE_pie['Verwaltungseinheit']==273125]

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²),code_len
621,Hausen,SRID=31468;POINT (4500554.730811018 5413337.21...,273125,Gemeinde,11.0,0.0,0.0,76.6,23.4,0.0,0.0,0.0,13851.0,4220.0,0.0,18071.0,2190.0,34.6,6


In [29]:
set(EE_pie['Name'].unique()) ^ set(energy['Name'].unique())

{'Mittelfranken',
 'Niederbayern',
 'Oberbayern',
 'Oberfranken',
 'Oberpfalz',
 'Schwaben',
 'Unterfranken'}

In [58]:
mobile[mobile['Verwaltungsebene']=='3 - Kreis']['Name']

1        Kreisfreie Stadt Ingolstadt
2           Kreisfreie Stadt München
3         Kreisfreie Stadt Rosenheim
4                Landkreis Altötting
5     Landkreis Berchtesgadener Land
                   ...              
92       Landkreis Lindau (Bodensee)
93               Landkreis Ostallgäu
94             Landkreis Unterallgäu
95              Landkreis Donau-Ries
96              Landkreis Oberallgäu
Name: Name, Length: 96, dtype: object

In [59]:
Kreise = EE_pie[EE_pie['Verwaltungsebene']=='Landkreis']['Name'].unique().tolist()

In [38]:
EE_pie[EE_pie['Verwaltungsebene']=='Landkreis']['Name'].nunique()

96

In [55]:
mobile[mobile['Verwaltungsebene']=='3 - Kreis']['Name'].nunique()

96

In [47]:
set(EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']['Name'].unique()) ^ set(mobile[mobile['Verwaltungsebene']=='4 - Gemeinde']['Name'].unique())

{'Schaufling',
 'Sulzthal',
 'Seeon-Seebruck',
 'Gemeinde Wiesenttal',
 'Thierhaupten',
 'Ramerberg',
 'Weitnau',
 'Attenhofen',
 'Wiesau',
 'Stadt Eichstätt',
 'Bad Tölz',
 'Floß',
 'Mainleus',
 'Gemeinde Megesheim',
 'Gemeinde Marktrodach',
 'Gemeinde Frasdorf',
 'Gemeinde Erharting',
 'Gemeinde Gebenbach',
 'Saulgrub',
 'Gemeindefreies Gebiet Forst Schmalwasser-Nord',
 'Giebelstadt',
 'Elfershausen',
 'Sand a.Main',
 'Gemeinde Salching',
 'Gemeinde Kronburg',
 'Ismaning',
 'Halsbach',
 'Bezirk München Ludwigsvorstadt-Isarvorstadt',
 'Gemeindefreies Gebiet Abenberger Wald',
 'Gemeinde Walting',
 'Gemeinde Mainleus',
 'Otterfing',
 'Ascha',
 'Gemeinde Heroldsbach',
 'Gemeindefreies Gebiet Rotter Forst-Nord',
 'Stadt Stadtbergen',
 'Gemeindefreies Gebiet Buckenhofer Forst',
 'Oberottmarshausen',
 'Bezirk München Laim',
 'Gemeinde Innernzell',
 'Illschwang',
 'Gemeinde Wallerfing',
 'Stadt Regensburg',
 'Mallersdorf-Pfaffenberg',
 'Tschirn',
 'Stadt Aschaffenburg',
 'Gemeinde Eitting',


In [54]:
mobile['AGS']

0             9
1          9161
2          9162
3          9163
4          9171
         ...   
2345    9780144
2346    9780145
2347    9780146
2348    9780147
2349    9780451
Name: AGS, Length: 2350, dtype: int64

In [49]:
EE_pie.head()

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²)
0,Oberbayern,SRID=31468;POINT (4481899.892788588 5329110.73...,100000,Regierungsbezirk,unbekannt,44.8,3.5,30.5,19.7,1.6,4957694.0,387394.0,3374166.0,2181533.0,173796.0,1.1074584E7,4729243.0,17534.0
1,Ingolstadt,SRID=31468;POINT (4457728.797577065 5403236.78...,161000,Gemeinde,10.0,44.6,0.0,27.1,28.3,0.0,90154.0,0.0,54769.0,57223.0,0.0,202146.0,138016.0,133.0
2,Ingolstadt,SRID=31468;POINT (4455613.551440659 5402131.15...,161000,Landkreis,unbekannt,44.6,0.0,27.1,28.3,0.0,90154.0,0.0,54769.0,57223.0,0.0,202146.0,138016.0,133.0
3,München,SRID=31468;POINT (4468515.3765645055 5333310.3...,162000,Gemeinde,14.0,45.1,4.6,40.8,9.5,0.0,64920.0,6586.0,58726.0,13709.0,0.0,143940.0,1487708.0,311.0
4,München (Stadt),SRID=31468;POINT (4466431.185606031 5335113.93...,162000,Landkreis,unbekannt,45.1,4.6,40.8,9.5,0.0,64920.0,6586.0,58726.0,13709.0,0.0,143940.0,1487708.0,311.0


In [62]:
len(Kreise)

96

In [74]:
Kreise_reduced = [x for x in Kreise if 'Stadt' not in x]

In [75]:
Gemeinden = EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']['Name'].unique().tolist()

In [77]:
Gemeinden

['Ingolstadt',
 'München',
 'Rosenheim',
 'Altötting',
 'Burghausen',
 'Burgkirchen a.d.Alz',
 'Emmerting',
 'Erlbach',
 'Feichten a.d.Alz',
 'Garching a.d.Alz',
 'Haiming',
 'Halsbach',
 'Kastl',
 'Kirchweidach',
 'Marktl',
 'Mehring',
 'Neuötting',
 'Perach',
 'Pleiskirchen',
 'Reischach',
 'Stammham',
 'Teising',
 'Töging a.Inn',
 'Tüßling',
 'Tyrlaching',
 'Unterneukirchen',
 'Winhöring',
 'Ainring',
 'Anger',
 'Bad Reichenhall',
 'Bayerisch Gmain',
 'Berchtesgaden',
 'Bischofswiesen',
 'Freilassing',
 'Laufen',
 'Marktschellenberg',
 'Piding',
 'Ramsau b.Berchtesgaden',
 'Saaldorf-Surheim',
 'Schneizlreuth',
 'Schönau a.Königssee',
 'Teisendorf',
 'Eck',
 'Schellenberger Forst',
 'Bad Heilbrunn',
 'Bad Tölz',
 'Benediktbeuern',
 'Bichl',
 'Dietramszell',
 'Egling',
 'Eurasburg',
 'Gaißach',
 'Geretsried',
 'Greiling',
 'Icking',
 'Jachenau',
 'Kochel a.See',
 'Königsdorf',
 'Lenggries',
 'Münsing',
 'Reichersbeuern',
 'Sachsenkam',
 'Schlehdorf',
 'Wackersberg',
 'Wolfratshausen',

In [80]:
[x for x in Gemeinden if 'Altendorf' in x]

['Altendorf']

In [81]:
EE_pie[EE_pie['Name'].str.contains('Altendorf')]

,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²)
1011,Altendorf,SRID=31468;POINT (4520428.055461742 5474248.08...,376112,Gemeinde,6.0,8.9,0.0,24.1,67.0,0.0,571.0,0.0,1541.0,4287.0,0.0,6399.0,843.0,23.2
1082,Altendorf,SRID=31468;POINT (4428874.116217541 5518322.88...,471111,Gemeinde,4.0,0.0,0.0,29.7,70.3,0.0,0.0,0.0,1482.0,3516.0,0.0,4998.0,2129.0,8.7


In [83]:
mobile[mobile['Name'].str.contains('Altendorf')]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,5G DSS.1,5G.1,2G.2,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3
1085,9376112,Gemeinde Altendorf,4 - Gemeinde,Freistaat Bayern,Landkreis Schwandorf,3 - ländlich,100.0,98.23,87.81,69.66,...,0.0,2.46,99.39,94.08,69.32,69.32,98.4,93.95,66.42,0.00
1149,9471111,Gemeinde Altendorf,4 - Gemeinde,Freistaat Bayern,Landkreis Bamberg,2 - halbstädtisch,100.0,100.00,92.32,99.41,...,0.0,98.94,100.00,99.76,91.84,97.40,100.0,97.87,89.24,24.59


In [86]:
mobile['AGS']

0             9
1          9161
2          9162
3          9163
4          9171
         ...   
2345    9780144
2346    9780145
2347    9780146
2348    9780147
2349    9780451
Name: AGS, Length: 2350, dtype: int64

In [87]:
EE_Gemeinde = EE_pie[EE_pie['Verwaltungsebene']=='Gemeinde']

In [101]:
mobile['AGS_2'] = mobile['AGS'].apply(lambda x: int(str(x)[1:]))
mobile.head()

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,5G.1,2G.2,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,AGS_2
0,9161,Kreisfreie Stadt Ingolstadt,3 - Kreis,Freistaat Bayern,Kreisfreie Stadt Ingolstadt,NaN,100.00,99.96,91.70,98.74,...,94.07,100.00,97.84,86.52,91.29,100.00,99.45,86.16,56.53,161
1,9162,Kreisfreie Stadt München,3 - Kreis,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.00,100.00,99.80,99.81,...,98.43,100.00,99.90,99.16,99.24,100.00,100.00,94.99,65.23,162
2,9163,Kreisfreie Stadt Rosenheim,3 - Kreis,Freistaat Bayern,Kreisfreie Stadt Rosenheim,NaN,100.00,100.00,99.70,99.65,...,97.92,100.00,99.86,96.27,97.76,100.00,100.00,90.68,77.68,163
3,9171,Landkreis Altötting,3 - Kreis,Freistaat Bayern,Landkreis Altötting,NaN,99.99,98.72,70.66,84.90,...,68.35,99.70,91.08,50.75,64.62,99.76,89.22,53.62,16.62,171
4,9172,Landkreis Berchtesgadener Land,3 - Kreis,Freistaat Bayern,Landkreis Berchtesgadener Land,NaN,91.69,75.27,38.15,55.13,...,43.62,85.67,62.33,29.65,34.82,70.89,52.54,26.34,6.50,172


In [108]:
LK_dict = mobile[['AGS_2','Name']].set_index('AGS_2').to_dict()['Name']

In [103]:
mobile[mobile['AGS']==9161000]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,5G.1,2G.2,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,AGS_2
96,9161000,Stadt Ingolstadt,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt Ingolstadt,1 - städtisch,100.0,99.96,91.7,98.74,...,94.07,100.0,97.84,86.52,91.29,100.0,99.45,86.16,56.53,161000


In [109]:
EE_Gemeinde['AGS'] = EE_Gemeinde['Verwaltungseinheit'].map(LK_dict)
EE_Gemeinde.head()

C:\Users\FabianLandua\AppData\Local\Temp\ipykernel_7148\3957486010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EE_Gemeinde['AGS'] = EE_Gemeinde['Verwaltungseinheit'].map(LK_dict)


,Name,Geometrie (EWKT),Verwaltungseinheit,Verwaltungsebene,Planungsregion,Anteil Wasserkraft an Stromproduktion EE (%),Anteil Wind an Stromproduktion EE (%),Anteil PV an Stromproduktion EE (%),Anteil Biomasse an Stromproduktion EE (%),Anteil Geothermie an Stromproduktion EE (%),Stromproduktion Wasserkraft 2022 (MWh/a),Stromproduktion Wind 2022 (MWh/a),Stromproduktion PV 2022 (MWh/a),Stromproduktion Biomasse 2022 (MWh/a),Stromproduktion Geothermie 2022 (MWh/a),Stromproduktion EE 2022 (MWh/a),Einwohnerzahl,Fläche (km²),AGS
1,Ingolstadt,SRID=31468;POINT (4457728.797577065 5403236.78...,161000,Gemeinde,10.0,44.6,0.0,27.1,28.3,0.0,90154.0,0.0,54769.0,57223.0,0.0,202146.0,138016.0,133.0,Stadt Ingolstadt
3,München,SRID=31468;POINT (4468515.3765645055 5333310.3...,162000,Gemeinde,14.0,45.1,4.6,40.8,9.5,0.0,64920.0,6586.0,58726.0,13709.0,0.0,143940.0,1487708.0,311.0,NaN
5,Rosenheim,SRID=31468;POINT (4509448.34360067 5301963.408...,163000,Gemeinde,18.0,11.6,0.0,15.1,73.3,0.0,11109.0,0.0,14526.0,70388.0,0.0,96023.0,63508.0,37.2,Stadt Rosenheim
8,Altötting,SRID=31468;POINT (4550547.681981055 5343559.06...,171111,Gemeinde,18.0,0.74,0.0,62.5,36.7,0.0,166.0,0.0,14086.0,8276.0,0.0,22528.0,12983.0,23.1,Stadt Altötting
9,Burghausen,SRID=31468;POINT (4561938.892540987 5337092.39...,171112,Gemeinde,18.0,92.7,0.0,5.7,1.7,0.0,230482.0,0.0,14106.0,4171.0,0.0,248759.0,18862.0,19.8,Stadt Burghausen


In [110]:
EE_Gemeinde.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2229 entries, 1 to 2331
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Name                                          2229 non-null   object 
 1   Geometrie (EWKT)                              2229 non-null   object 
 2   Verwaltungseinheit                            2229 non-null   int64  
 3   Verwaltungsebene                              2229 non-null   object 
 4   Planungsregion                                2229 non-null   object 
 5   Anteil Wasserkraft an Stromproduktion EE (%)  2229 non-null   object 
 6   Anteil Wind an Stromproduktion EE (%)         2229 non-null   object 
 7   Anteil PV an Stromproduktion EE (%)           2229 non-null   object 
 8   Anteil Biomasse an Stromproduktion EE (%)     2229 non-null   object 
 9   Anteil Geothermie an Stromproduktion EE (%)   2229 non-null   object

In [111]:
mobile[mobile['Name'].str.contains('München')]

,AGS,Name,Verwaltungsebene,Land,Kreis,Raumkategorie,2G,4G,5G DSS,5G,...,5G.1,2G.2,4G.2,5G DSS.2,5G.2,2G.3,4G.3,5G DSS.3,5G.3,AGS_2
1,9162,Kreisfreie Stadt München,3 - Kreis,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,99.80,99.81,...,98.43,100.00,99.90,99.16,99.24,100.00,100.00,94.99,65.23,162
16,9184,Landkreis München,3 - Kreis,Freistaat Bayern,Landkreis München,NaN,100.0,99.74,87.21,86.85,...,57.32,99.96,96.70,76.38,81.60,99.98,97.75,73.57,23.58,184
97,9162001,Bezirk München Altstadt-Lehel,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,89.52,162001
98,9162002,Bezirk München Ludwigsvorstadt-Isarvorstadt,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,92.20,162002
99,9162003,Bezirk München Maxvorstadt,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,99.77,162003
100,9162004,Bezirk München Schwabing-West,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,89.34,162004
101,9162005,Bezirk München Au-Haidhausen,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,99.06,82.08,162005
102,9162006,Bezirk München Sendling,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,97.19,162006
103,9162007,Bezirk München Sendling-Westpark,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,96.93,34.14,162007
104,9162008,Bezirk München Schwanthalerhöhe,4 - Gemeinde,Freistaat Bayern,Kreisfreie Stadt München,NaN,100.0,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,99.53,162008


In [2]:
files = os.listdir('../data/external/')

In [4]:
[x for x in files if 'bba' in x]

['bba_01_2024.xlsx', 'bba_06_2023.xlsx']

: 